In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import sem

<h3>This project is based on dummy data of an inventory management of a shoe shop</h3>
<p>Analysis is because the shop has so many shoes unsold in their inventory.</p>

<h2>Problem Statements</h2>
<ul>
    <li>How many Shoes are likely to be sold based on historical data?</li>
    <li>By how much does one shop outperform the other?</li>
</ul>

In [2]:
shop_df = pd.read_excel('inventory.xlsx')

In [3]:
shop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14970 entries, 0 to 14969
Data columns (total 15 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Unnamed: 0                                    0 non-null      float64
 1   Inferential statistics. Confidence intervals  14969 non-null  object 
 2   Unnamed: 2                                    14968 non-null  object 
 3   Unnamed: 3                                    14968 non-null  object 
 4   Unnamed: 4                                    14968 non-null  object 
 5   Unnamed: 5                                    14968 non-null  object 
 6   Unnamed: 6                                    14968 non-null  object 
 7   Unnamed: 7                                    14968 non-null  object 
 8   Unnamed: 8                                    14968 non-null  object 
 9   Unnamed: 9                                    14968 non-null 

In [4]:
shop_df.head()

,Unnamed: 0,Inferential statistics. Confidence intervals,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,Al Bundy's shoe shop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,InvoiceNo,Date,Country,ProductID,Shop,Gender,Size (US),Size (Europe),Size (UK),UnitPrice,Discount,NaN,Month,SalePrice
3,NaN,52389,2014-01-01 01:06:24,United Kingdom,2152,UK2,Male,11,44,10.5,159,0,2014.0,1,159
4,NaN,52390,2014-01-01 03:44:48,United States,2230,US15,Male,11.5,44-45,11,199,0.2,2014.0,1,159.2


In [5]:
# As we can see from the dataframe, the columns are unnamed and rows 0 and 1 should be dropped
shop_df.drop(index=[0,1], columns=['Unnamed: 0'], inplace=True)
shop_df.loc[2,'Unnamed: 12'] = 'Year'

In [6]:
#Renaming the columns
new_columns = [i for i in shop_df.loc[2,:]]
shop_df.columns = new_columns

In [7]:
# Dropping unwanted row, resetting index, creating a new column for future use
shop_df.drop(2,axis=0, inplace=True)
shop_df.reset_index(drop=True, inplace=True)
shop_df['Sales'] = shop_df['SalePrice']

In [8]:
shop_df.head()

,InvoiceNo,Date,Country,ProductID,Shop,Gender,Size (US),Size (Europe),Size (UK),UnitPrice,Discount,Year,Month,SalePrice,Sales
0,52389,2014-01-01 01:06:24,United Kingdom,2152,UK2,Male,11,44,10.5,159,0,2014,1,159,159
1,52390,2014-01-01 03:44:48,United States,2230,US15,Male,11.5,44-45,11,199,0.2,2014,1,159.2,159.2
2,52391,2014-01-01 09:01:36,Canada,2160,CAN7,Male,9.5,42-43,9,149,0.2,2014,1,119.2,119.2
3,52392,2014-01-01 09:16:00,United States,2234,US6,Female,9.5,40,7.5,159,0,2014,1,159,159
4,52393,2014-01-01 13:35:12,United Kingdom,2222,UK4,Female,9,39-40,7,159,0,2014,1,159,159


### 1. How many shoes are likely to be sold based on historical data?

<b>We should create a new dataframe selecting the Males and United States columns and setting the US size as our index as the rest of the sizes are conversions based on  the country and the problem we are trying to solve will require us to use only either the male or female sex/gender.</b>
<p><b>This DataFrame will serve as a Frequency Distribution Table</b></p>

In [9]:
m_c_df = shop_df[(shop_df['Country']=='United States') & (shop_df['Gender']=='Male')].set_index(['Size (US)'])

#Dropping columns that won't be useful in our analysis

m_c_df.drop(columns=['InvoiceNo', 'Date', 'ProductID', 'Shop', 'Size (Europe)',  
                     'Size (UK)', 'UnitPrice', 'SalePrice'], axis=1, inplace=True)

In [10]:
#We'll use the data for year 2016
m_c_df = m_c_df[m_c_df['Year']==2016]


m_c_df = m_c_df.groupby(['Month', 'Size (US)']).count().\
                         drop(columns=['Country', 'Gender', 'Discount', 'Year']).\
                         sort_values(by='Sales').unstack(level=0)
m_c_df # The dataframe shows amount of shoes sold for every size of men shoe monthly

Sales                                                              \
Month        1     2     3     4     5     6     7     8     9     10    11   
Size (US)                                                                     
6.0         4.0   1.0   3.0   1.0   3.0   3.0   3.0   4.0   3.0   7.0   3.0   
6.5         3.0   2.0   NaN   1.0   NaN   NaN   1.0   7.0   2.0   1.0   2.0   
7.0         NaN   NaN   1.0   NaN   6.0   4.0   4.0   2.0   3.0   NaN   NaN   
7.5         3.0   2.0   3.0   1.0   7.0   NaN   7.0   3.0   4.0   6.0   1.0   
8.0         7.0   9.0   7.0   3.0  12.0   2.0   9.0   4.0   7.0   5.0   2.0   
8.5        12.0  12.0   8.0   8.0  15.0   9.0  17.0  17.0   6.0   9.0  10.0   
9.0        17.0  13.0  13.0  11.0  21.0  22.0  25.0  30.0  26.0  25.0  13.0   
9.5        19.0  25.0  27.0  24.0  26.0  33.0  25.0  47.0  31.0  44.0  37.0   
10.0       17.0  26.0  26.0  19.0  16.0  31.0  25.0  24.0  23.0  31.0  15.0   
10.5       13.0  16.0  22.0  14.0  28.0  19.0  18.0  15.0  19.0  21.0  16.0   
11.0        5.0  16.0  13.0  10.0  10.0  11.0  15.0   8.0   9.0   7.0   6.0   
11.5        4.0   3.0   6.0   3.0   3.0   5.0   6.0   4.0   5.0  12.0  13.0   
12.0        3.0   NaN   NaN   4.0   4.0   4.0   3.0  12.0   4.0   9.0   2.0   
13.0        1.0   1.0   2.0   NaN   3.0   2.0   1.0   NaN   NaN   4.0   3.0   
14.0        2.0   6.0   3.0   3.0   5.0   3.0   2.0   1.0   NaN   1.0   2.0   
15.0        NaN   NaN   NaN   1.0   1.0   NaN   4.0   NaN   NaN   NaN   NaN   

                 
Month        12  
Size (US)        
6.0         NaN  
6.5         1.0  
7.0         NaN  
7.5         1.0  
8.0         6.0  
8.5         6.0  
9.0        10.0  
9.5        26.0  
10.0       20.0  
10.5       10.0  
11.0        7.0  
11.5        5.0  
12.0        1.0  
13.0        2.0  
14.0        1.0  
15.0        2.0

In [11]:
m_c_df.fillna(0, inplace=True)

In [12]:
m_c_df.head()

Sales                                                        
Month        1    2    3    4     5    6    7    8    9    10   11   12
Size (US)                                                              
6.0         4.0  1.0  3.0  1.0   3.0  3.0  3.0  4.0  3.0  7.0  3.0  0.0
6.5         3.0  2.0  0.0  1.0   0.0  0.0  1.0  7.0  2.0  1.0  2.0  1.0
7.0         0.0  0.0  1.0  0.0   6.0  4.0  4.0  2.0  3.0  0.0  0.0  0.0
7.5         3.0  2.0  3.0  1.0   7.0  0.0  7.0  3.0  4.0  6.0  1.0  1.0
8.0         7.0  9.0  7.0  3.0  12.0  2.0  9.0  4.0  7.0  5.0  2.0  6.0

<h3 style="text-align:center">To solve the problem, we'll have to use the Formula for CI for a single population and variance is unknown with 95% Confidence Interval</h3>

In [13]:
#FIRST WE'LL GET THE MEAN SIZE OF EACH SHOE SIZE SOLD IN EACH MONTH AND ASSIGN IT TO A MEAN COLUMN

m_c_df['Mean'] = [round(m_c_df.iloc[i,:].mean(),2) for i in range(len(m_c_df))]
m_c_df.head(2)

Sales                                                         Mean
Month         1    2    3    4    5    6    7    8    9   10   11   12      
Size (US)                                                                   
6.0         4.0  1.0  3.0  1.0  3.0  3.0  3.0  4.0  3.0  7.0  3.0  0.0  2.92
6.5         3.0  2.0  0.0  1.0  0.0  0.0  1.0  7.0  2.0  1.0  2.0  1.0  1.67

In [14]:
n = 12# Our Sample size is data  for last 12 months(YEAR 2016)
# Our Population Variance is unknown, therefore we'll be using the t-statistic
df = n-1 #degrees of freedom
alpha = 0.05 # significance level
t = stats.t.ppf(1-alpha/2, df) # t-critical value

In [15]:
# Create a list containing our lower and upper CI for each shoe size
ci_list = []
pairs = [] #Shoes that are sold the most

for i in range(len(m_c_df)):
    data = m_c_df.iloc[i,:]
    s = np.std(data, ddof=1)# Sample standard deviation
    lower = round(np.mean(data)-(t * s/np.sqrt(n)),2) # Lower interval
    upper = round(np.mean(data)+(t * s/np.sqrt(n)),2) # Upper interval
    ci_list.append(str(lower) + " - " + str(upper))
    re_shoes = round(np.mean(data)+(t * s/np.sqrt(n))) # Recommended amount of each pair to be sold
    pairs.append(re_shoes)

In [16]:
m_c_df['95% CI'] = ci_list
m_c_df['Num_of_Pairs'] = pairs
m_c_df

Sales                                                              \
Month         1     2     3     4     5     6     7     8     9    10    11   
Size (US)                                                                     
6.0         4.0   1.0   3.0   1.0   3.0   3.0   3.0   4.0   3.0   7.0   3.0   
6.5         3.0   2.0   0.0   1.0   0.0   0.0   1.0   7.0   2.0   1.0   2.0   
7.0         0.0   0.0   1.0   0.0   6.0   4.0   4.0   2.0   3.0   0.0   0.0   
7.5         3.0   2.0   3.0   1.0   7.0   0.0   7.0   3.0   4.0   6.0   1.0   
8.0         7.0   9.0   7.0   3.0  12.0   2.0   9.0   4.0   7.0   5.0   2.0   
8.5        12.0  12.0   8.0   8.0  15.0   9.0  17.0  17.0   6.0   9.0  10.0   
9.0        17.0  13.0  13.0  11.0  21.0  22.0  25.0  30.0  26.0  25.0  13.0   
9.5        19.0  25.0  27.0  24.0  26.0  33.0  25.0  47.0  31.0  44.0  37.0   
10.0       17.0  26.0  26.0  19.0  16.0  31.0  25.0  24.0  23.0  31.0  15.0   
10.5       13.0  16.0  22.0  14.0  28.0  19.0  18.0  15.0  19.0  21.0  16.0   
11.0        5.0  16.0  13.0  10.0  10.0  11.0  15.0   8.0   9.0   7.0   6.0   
11.5        4.0   3.0   6.0   3.0   3.0   5.0   6.0   4.0   5.0  12.0  13.0   
12.0        3.0   0.0   0.0   4.0   4.0   4.0   3.0  12.0   4.0   9.0   2.0   
13.0        1.0   1.0   2.0   0.0   3.0   2.0   1.0   0.0   0.0   4.0   3.0   
14.0        2.0   6.0   3.0   3.0   5.0   3.0   2.0   1.0   0.0   1.0   2.0   
15.0        0.0   0.0   0.0   1.0   1.0   0.0   4.0   0.0   0.0   0.0   0.0   

                  Mean         95% CI Num_of_Pairs  
Month        12                                     
Size (US)                                           
6.0         0.0   2.92     1.83 - 4.0          4.0  
6.5         1.0   1.67     0.5 - 2.84          3.0  
7.0         0.0   1.67    0.39 - 2.95          3.0  
7.5         1.0   3.17     1.7 - 4.63          5.0  
8.0         6.0   6.08    4.22 - 7.94          8.0  
8.5         6.0  10.75   8.39 - 13.11         13.0  
9.0        10.0  18.83  14.68 - 22.98         23.0  
9.5        26.0  30.33  25.18 - 35.49         35.0  
10.0       20.0  22.75  19.45 - 26.05         26.0  
10.5       10.0  17.58   14.7 - 20.47         20.0  
11.0        7.0   9.75   7.62 - 11.88         12.0  
11.5        5.0   5.75    3.72 - 7.78          8.0  
12.0        1.0   3.83     1.7 - 5.97          6.0  
13.0        2.0   1.58    0.79 - 2.38          2.0  
14.0        1.0   2.42    1.36 - 3.47          3.0  
15.0        2.0   0.67   -0.08 - 1.42          1.0

<b>We can see that the most shoes that were sold are between 8.5 and 11.5. Our recommendation will be to buy quantity in respect to our upper bound which is the Num_of _Pairs column</b>

### 2.  By how much does one shop out perform the other?

<h4>For this problem, we'll be using the two shops GER1 and GER2 and also the women shoe sizes this time. It'll also be a 95% interval</h4>

In [17]:
shop_df.columns

Index(['InvoiceNo', 'Date', 'Country', 'ProductID', 'Shop', 'Gender',
       'Size (US)', 'Size (Europe)', 'Size (UK)', 'UnitPrice', 'Discount',
       'Year', 'Month', 'SalePrice', 'Sales'],
      dtype='object')

In [18]:
data1 = shop_df[(shop_df['Shop']=='GER1') & (shop_df['Gender']=='Female')]
data2 = shop_df[(shop_df['Shop']=='GER2') & (shop_df['Gender']=='Female')]

In [19]:
data1.head()

,InvoiceNo,Date,Country,ProductID,Shop,Gender,Size (US),Size (Europe),Size (UK),UnitPrice,Discount,Year,Month,SalePrice,Sales
15,52401,2014-01-03 02:32:48,Germany,2197,GER1,Female,8.5,39,6.5,179,0.2,2014,1,143.2,143.2
49,52429,2014-01-07 09:59:12,Germany,2239,GER1,Female,9.5,40,7.5,159,0.5,2014,1,79.5,79.5
180,52535,2014-01-25 13:20:48,Germany,2197,GER1,Female,8.5,39,6.5,179,0.2,2014,1,143.2,143.2
192,52546,2014-01-26 21:59:12,Germany,2227,GER1,Female,9,39-40,7,149,0.1,2014,1,134.1,134.1
214,52566,2014-01-31 13:49:36,Germany,2165,GER1,Female,10.5,41,8.5,179,0.2,2014,1,143.2,143.2


In [20]:
data2.head()

,InvoiceNo,Date,Country,ProductID,Shop,Gender,Size (US),Size (Europe),Size (UK),UnitPrice,Discount,Year,Month,SalePrice,Sales
6,52395,2014-01-02 01:06:24,Germany,2200,GER2,Female,9,39-40,7,179,0,2014,1,179,179
33,52414,2014-01-05 02:32:48,Germany,2239,GER2,Female,8.5,39,6.5,129,0.5,2014,1,64.5,64.5
47,52427,2014-01-07 09:44:48,Germany,2242,GER2,Female,7,37-38,5,189,0.1,2014,1,170.1,170.1
54,52434,2014-01-08 09:59:12,Germany,2238,GER2,Female,9.5,40,7.5,189,0,2014,1,189,189
105,52475,2014-01-16 17:11:12,Germany,2242,GER2,Female,7,37-38,5,189,0.1,2014,1,170.1,170.1


In [21]:
# We'll also be using the data from year 2016
data1 = data1[data1['Year']==2016]
data2 = data2[data2['Year']==2016]

<b>Our Assumption will be that same people don't buy shoes from different shops in the same year, therefore the two samples are independent and we don't know the sample variance. Also, given this is the same market in the same country we can assume it is equal.</b>

In [22]:
data1.set_index(['Size (US)'])

,InvoiceNo,Date,Country,ProductID,Shop,Gender,Size (Europe),Size (UK),UnitPrice,Discount,Year,Month,SalePrice,Sales
Size (US),,,,,,,,,,,,,,
8.5,59199,2016-01-01 15:30:24,Germany,2173,GER1,Female,39,6.5,149,0.2,2016,1,119.2,119.2
7.5,59202,2016-01-02 02:47:12,Germany,2205,GER1,Female,38,5.5,159,0,2016,1,159,159
8.5,59208,2016-01-02 09:59:12,Germany,2195,GER1,Female,39,6.5,179,0,2016,1,179,179
8.0,59215,2016-01-02 23:25:36,Germany,2231,GER1,Female,38-39,6,189,0.3,2016,1,132.3,132.3
6.5,59236,2016-01-04 08:18:24,Germany,2229,GER1,Female,37,4.5,149,0.5,2016,1,74.5,74.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7.0,65733,2016-12-28 19:20:48,Germany,2242,GER1,Female,37-38,5,189,0.3,2016,12,132.3,132.3
9.5,65742,2016-12-29 11:40:00,Germany,2188,GER1,Female,40,7.5,189,0,2016,12,189,189
7.5,65768,2016-12-31 04:42:24,Germany,2205,GER1,Female,38,5.5,159,0,2016,12,159,159


In [23]:
#Dropping columns that won't be useful in our analysis

data1.drop(columns=['InvoiceNo', 'Date', 'ProductID', 'Shop', 'Size (Europe)',  
                     'Size (UK)', 'UnitPrice', 'SalePrice'], axis=1, inplace=True)

data2.drop(columns=['InvoiceNo', 'Date', 'ProductID', 'Shop', 'Size (Europe)',  
                     'Size (UK)', 'UnitPrice', 'SalePrice'], axis=1, inplace=True)

In [24]:
data1 = data1.groupby(['Month', 'Size (US)']).count().\
                         drop(columns=['Country', 'Gender', 'Discount', 'Year']).\
                         sort_values(by='Sales').unstack(level=0)


data2 = data2.groupby(['Month', 'Size (US)']).count().\
                         drop(columns=['Country', 'Gender', 'Discount', 'Year']).\
                         sort_values(by='Sales').unstack(level=0)

In [25]:
data1.fillna(0, inplace=True)
data2.fillna(0, inplace=True)

In [26]:
data1.head()

Sales                                                       
Month        1    2    3    4    5    6    7    8    9    10   11   12
Size (US)                                                             
4.5         0.0  0.0  0.0  0.0  1.0  3.0  0.0  0.0  0.0  0.0  1.0  0.0
5.0         0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0
5.5         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
6.0         0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
6.5         3.0  3.0  1.0  2.0  1.0  0.0  2.0  0.0  2.0  1.0  3.0  4.0

In [27]:
data2.head()

Sales                                                       
Month        1    2    3    4    5    6    7    8    9    10   11   12
Size (US)                                                             
4.5         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
5.0         0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0
5.5         0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  2.0  0.0  1.0
6.0         0.0  1.0  3.0  1.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
6.5         2.0  0.0  2.0  1.0  1.0  2.0  0.0  1.0  2.0  1.0  3.0  0.0

In [28]:
len(data2.columns)

12

In [29]:
alpha = 0.05 # significance level
n1,n2 = len(data1.columns), len(data2.columns) # sample sizes 
df = n1 + n2 - 2 # degrees of freedom as we are dealing with 2 samples
t = stats.t.ppf(1-alpha/2, df) # t-critical value

In [30]:
ci_list = [] # List to contain our intervals

for a, b in zip(range(len(data1)), range(len(data2))):
    x1 = data1.iloc[a,:]
    x2 = data2.iloc[b,:]
    s1,s2 = np.var(x1, ddof=1), np.var(x2, ddof=1) # sample variances
    s = np.sqrt(((n1-1)*s1 + (n2-1)*s2)/(n1+n2-2)) # pooled(weighted average) variance
    lower = round((np.mean(x1) - np.mean(x2)) - t * np.sqrt(1/len(x1) + 1/len(x2))*s, 2)
    upper = round((np.mean(x1) - np.mean(x2)) + t * np.sqrt(1/len(x1) + 1/len(x2))*s, 2)
    ci_list.append(str(lower) + '   ' + str(upper))

In [31]:
ci_list # CONFIDENCE INTERVAL FOR THE TWO SHOPS

['-0.23   0.9',
 '-0.49   0.49',
 '-0.68   0.18',
 '-1.11   0.27',
 '-0.37   1.54',
 '-1.57   0.91',
 '-4.83   0.66',
 '-4.21   1.71',
 '-0.72   3.55',
 '-2.12   1.96',
 '-0.79   1.96',
 '-0.72   0.89',
 '-0.16   1.66',
 '-0.54   0.21',
 '-1.27   0.6',
 '-0.65   0.32']

<b><i>NOTE: All the Confidence Intervals starts in negatives and end in positives.</i> We can not conclude that one shop sells significantly more shoes than the other for any size. This shows that GER1 is more likely to sell some shoe sizes than GER2 and vice-versa.<p>Insight: these two shops are balanced in terms of sales.<br/>On average, they'll move together. They are predicted to remain identical.</p></b>

In [32]:
shop_df.head()

,InvoiceNo,Date,Country,ProductID,Shop,Gender,Size (US),Size (Europe),Size (UK),UnitPrice,Discount,Year,Month,SalePrice,Sales
0,52389,2014-01-01 01:06:24,United Kingdom,2152,UK2,Male,11,44,10.5,159,0,2014,1,159,159
1,52390,2014-01-01 03:44:48,United States,2230,US15,Male,11.5,44-45,11,199,0.2,2014,1,159.2,159.2
2,52391,2014-01-01 09:01:36,Canada,2160,CAN7,Male,9.5,42-43,9,149,0.2,2014,1,119.2,119.2
3,52392,2014-01-01 09:16:00,United States,2234,US6,Female,9.5,40,7.5,159,0,2014,1,159,159
4,52393,2014-01-01 13:35:12,United Kingdom,2222,UK4,Female,9,39-40,7,159,0,2014,1,159,159


In [34]:
pd.crosstab(shop_df['Size (US)'], shop_df['Month'])

Month,1,2,3,4,5,6,7,8,9,10,11,12
Size (US),,,,,,,,,,,,
4.5,1,8,3,0,2,17,5,3,2,5,1,4
5.0,5,0,1,0,3,2,6,7,10,1,0,1
5.5,5,3,0,6,1,3,6,4,5,5,8,17
6.0,12,24,10,24,23,11,4,15,15,30,10,8
6.5,33,24,25,30,24,33,42,35,27,22,23,24
7.0,26,26,36,45,44,57,40,42,63,59,46,53
7.5,87,72,67,63,76,91,83,68,107,107,71,59
8.0,135,143,127,128,136,118,162,148,138,169,128,124
8.5,107,104,124,122,169,156,178,178,145,164,142,127
